<h2>Animation</h2>

In [ ]:
import numpy as np
import matplotlib.patches as pat
from matplotlib.collections import PatchCollection
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import animation
import matplotlib.gridspec as gridspec
import h5py


plt.rcParams['animation.ffmpeg_path'] = "C:/ffmpeg/bin/ffmpeg.exe"
FFwriter = animation.FFMpegWriter()

In [ ]:
# Extract info from the file
f = h5py.File('.\Data\Simulation_L=20_W=10_steps=10000_la=0.20_ls=0.06.hdf5', 'r' )
L, W, steps, la, ls = f['L_W_steps_la_ls_ln_lfin_ltin'][0:5,0]
N=f['r'].shape[1]
steps = f['r'].shape[2]

#Choose number of frames
nframes = 100
dt = steps//nframes

In [ ]:
# Create the figure and axis
fig = plt.figure()

axcircles = plt.subplot2grid((2,2), (0,0), rowspan = 2)
axparam = plt.subplot2grid((2,2), (0,1))
axrmean = plt.subplot2grid((2,2), (1,1))

axcircles.set_title('Particles')
axparam.set_title('order parameter')
axrmean.set_title('r mean')

axcircles.set_xlabel('x')
axcircles.set_ylabel('y')

axparam.set_xlabel('time')
axparam.set_ylabel('\phi')

axrmean.set_xlabel('x')
axrmean.set_ylabel('y')

plt.tight_layout()

particles = []
particle_col = []

# colors of the particles depending on the amount of overlap
colormap = plt.get_cmap('jet')
colors = f['overlap'][:,0]*0.5

# plotting the particles for t=0
for ii in range(f['r'].shape[1]):
    patch = plt.Circle(f['r'][:,ii,0], f['size'][ii,0],  facecolor = colormap(colors[ii]), edgecolor = colormap(colors[ii]), 
                       alpha = 0.5)
    particles.append(patch)
    arrow = pat.FancyArrow(f['r'][0,ii,0],f['r'][1,ii,0], np.cos(f['psi'][ii,0]), np.sin(f['psi'][ii,0]))
    particles.append(arrow)
particle_col = PatchCollection(particles,match_original = True)
axcircles.set_xlim(f['r_mean'][0,0]- 1.5*np.max((W,L)),f['r_mean'][0,0]+ 1.5*np.max((W,L)))
axcircles.set_ylim(f['r_mean'][1,0]- 1.5*np.max((W,L)),f['r_mean'][1,0]+ 1.5*np.max((W,L)))
axcircles.add_collection(particle_col)

# Plotting the order parameter
orderparam = []
orderparam.append(f['order_param'][0])
axparam.plot(orderparam)

# plotting the center of mass 
r_meanx = []
r_meany = []
min_xm = np.min(f['r_mean'][0,0:nframes*dt:dt])
max_xm = np.max(f['r_mean'][0,0:nframes*dt:dt])
min_ym = np.min(f['r_mean'][1,0:nframes*dt:dt])
max_ym = np.max(f['r_mean'][1,0:nframes*dt:dt])
axrmean.set_xlim(min_xm-1,max_xm+1)
axrmean.set_ylim(min_ym-2,max_ym+2)
r_meanx.append(f['r_mean'][0,0])
r_meany.append(f['r_mean'][1,0])
axrmean.plot(r_meanx,r_meany)

print('Progress(frame) = ', end = ' ')
def animate(ti):
    # Clear the plots from previous frames 
    axcircles.clear()
    axparam.clear()
    axrmean.clear()
    
    axcircles.set_title('Particles')
    axparam.set_title('order parameter')
    axrmean.set_title('r mean')

    axcircles.set_xlabel('x')
    axcircles.set_ylabel('y')

    axparam.set_xlabel('time')
    axparam.set_ylabel('\phi')

    axrmean.set_xlabel('x')
    axrmean.set_ylabel('y')
    
    particles = []
    
    t = ti*dt # frames take steps in time of dt
    print(t//100, end =' ' )
    # Set color of particles according to amount of overlap
    colors = f['overlap'][:,t]*0.5
    if f['defect_counter'][t,0] > 0:
        colors[np.nonzero(f['defect'][:,t])] =1
        
    # Print particles at time t
    for ii in range(f['r'].shape[1]):
        patch = plt.Circle(f['r'][:,ii,t], f['size'][ii],  facecolor = colormap(colors[ii]), edgecolor = 
                           colormap(colors[ii]), alpha = 0.6)
        particles.append(patch)
    for ii in range(int(f['defect_counter'][t,0])):
        defect = plt.Circle(f['defect_r'][:,ii,t], f['size'][f['defect'][ii,t]], 
                            facecolor = colormap(0.5), alpha = 1)
        particles.append(defect)
    for ii in range(f['r'].shape[1]):
        arrow = pat.FancyArrow(f['r'][0,ii,t],f['r'][1,ii,t], np.cos(f['psi'][ii,t]), np.sin(f['psi'][ii,t]))
        particles.append(arrow)
    particles_col = PatchCollection(particles,match_original = True)
    axcircles.set_xlim(np.average(f['r'][0,:,t])- 1.5*np.max((W,L)),f['r_mean'][0,t]+ 1.5*np.max((W,L)))
    axcircles.set_ylim(f['r_mean'][1,t]- 1.5*np.max((W,L)),f['r_mean'][1,t]+ 1.5*np.max((W,L)))
    axcircles.add_collection(particles_col)
    # plot  order parameter

    orderparam.append(f['order_param'][t])
    axparam.plot(orderparam)

    # plot r_mean

    r_meanx.append(f['r_mean'][0,t])
    r_meany.append(f['r_mean'][1,t])
    axrmean.set_xlim(min_xm-1,max_xm+1)
    axrmean.set_ylim(min_ym-2,max_ym+2)
    axrmean.plot(r_meanx,r_meany)
       
    
    return 
ani = animation.FuncAnimation(fig, animate, frames = nframes, repeat = False )
ani.save(r'.\Figures\Simulation_L=%i_W=%i_steps=%i_la=%.2f_ls=%.2f.mp4' %(L,W,steps,la,ls) , writer = FFwriter, fps = 5)

 
plt.show()